### Data Processing

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

<br>

###  Load and Combine All Datasets

In [2]:
# Load price data (50 weapon cases)
df_prices = pd.read_csv('csv-files/page12345_steam_cases.csv')
df_prices['date'] = pd.to_datetime(df_prices['date'])

# Calculate daily average price across all items
daily_prices = df_prices.groupby(df_prices['date'].dt.date)['price'].mean().reset_index()
daily_prices.columns = ['Date', 'Average_Price']
daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])

print(f" Price data: {len(daily_prices)} days, {df_prices['item_name'].nunique()} items")
print(f" Date range: {daily_prices['Date'].min()} to {daily_prices['Date'].max()}")
print(f" Price range: ${daily_prices['Average_Price'].min():.2f} - ${daily_prices['Average_Price'].max():.2f}")

 Price data: 365 days, 50 items
 Date range: 2024-10-04 00:00:00 to 2025-10-03 00:00:00
 Price range: $1.24 - $2.43


In [3]:
# Load player count data
df_players = pd.read_csv('csv-files/csgo_daily_avg_players_last_year.csv')
df_players['Date'] = pd.to_datetime(df_players['Date'])

print(f" Player count data: {len(df_players)} days")
print(f" Date range: {df_players['Date'].min()} to {df_players['Date'].max()}")
print(f" Player range: {df_players['Average_Players'].min():,.0f} - {df_players['Average_Players'].max():,.0f}")

 Player count data: 365 days
 Date range: 2024-10-04 00:00:00 to 2025-10-03 00:00:00
 Player range: 870,136 - 1,862,531


In [4]:
# Load tournament event data
df_events = pd.read_csv('csv-files/csgo_tournament_events_daily_last_year.csv')
df_events['Date'] = pd.to_datetime(df_events['Date'])

print(f" Tournament data: {len(df_events)} days")
print(f" Date range: {df_events['Date'].min()} to {df_events['Date'].max()}")
print(f" Tournament days: {df_events['has_tournament'].sum()} ({df_events['has_tournament'].sum()/len(df_events)*100:.1f}%)")

 Tournament data: 365 days
 Date range: 2024-10-04 00:00:00 to 2025-10-03 00:00:00
 Tournament days: 178 (48.8%)


<br>



### Restructure for Item-Level Prediction

In [5]:
# Convert date to date only (remove time)
df_prices['date_only'] = df_prices['date'].dt.date
df_prices['date_only'] = pd.to_datetime(df_prices['date_only'])

# Group by item and date to get daily price per item
df_item_daily = df_prices.groupby(['item_name', 'date_only'])['price'].mean().reset_index()
df_item_daily.columns = ['item_name', 'Date', 'price']

# Merge with player count
df_item_daily = df_item_daily.merge(df_players, on='Date', how='left')

# Merge with tournament events
df_item_daily = df_item_daily.merge(df_events, on='Date', how='left')

# Fill missing tournament flags
df_item_daily['has_tournament'] = df_item_daily['has_tournament'].fillna(0).astype(int)

print(f" Item-level dataset created:")
print(f" Total records: {len(df_item_daily):,}")
print(f" Unique items: {df_item_daily['item_name'].nunique()}")
print(f" Days per item: {len(df_item_daily) // df_item_daily['item_name'].nunique()}")
print(f" Date range: {df_item_daily['Date'].min()} to {df_item_daily['Date'].max()}")
print(f"\n{df_item_daily.head(10)}")

 Item-level dataset created:
 Total records: 16,872
 Unique items: 50
 Days per item: 337
 Date range: 2024-10-04 00:00:00 to 2025-10-03 00:00:00

              item_name       Date  price  Average_Players  has_tournament
0  2020 RMR Challengers 2024-10-04  0.207        1322412.0               0
1  2020 RMR Challengers 2024-10-05  0.219        1363110.0               0
2  2020 RMR Challengers 2024-10-06  0.224        1462028.0               0
3  2020 RMR Challengers 2024-10-07  0.209        1363872.0               1
4  2020 RMR Challengers 2024-10-08  0.203        1400121.0               1
5  2020 RMR Challengers 2024-10-09  0.194        1395317.0               1
6  2020 RMR Challengers 2024-10-10  0.204        1346127.0               1
7  2020 RMR Challengers 2024-10-11  0.212        1368886.0               1
8  2020 RMR Challengers 2024-10-12  0.223        1498661.0               1
9  2020 RMR Challengers 2024-10-13  0.234        1457406.0               1


In [7]:
# Encode item names (simple numeric encoding)
# Create a mapping: item name -> unique integer
item_names = df_item_daily['item_name'].unique()
item_to_id = {name: idx for idx, name in enumerate(sorted(item_names))}
id_to_item = {idx: name for name, idx in item_to_id.items()}

df_item_daily['item_encoded'] = df_item_daily['item_name'].map(item_to_id)

# Show mapping (first 10 items)
print(f" Item name encoding (first 10):")
for i, (item, code) in enumerate(sorted(item_to_id.items())[:10]):
    print(f"   {code:2d}: {item}")
    
print(f"\n Total unique items encoded: {len(item_to_id)}")

# Save the mapping for later use (to decode predictions)
import pickle
with open('csv-files/item_mapping.pkl', 'wb') as f:
    pickle.dump({'item_to_id': item_to_id, 'id_to_item': id_to_item}, f)
print(f" Saved item mapping to: item_mapping.pkl")

 Item name encoding (first 10):
    0: 2020 RMR Challengers
    1: 2020 RMR Contenders
    2: 2020 RMR Legends
    3: Antwerp 2022 Challengers Autograph Capsule
    4: Antwerp 2022 Challengers Sticker Capsule
    5: Antwerp 2022 Contenders Sticker Capsule
    6: Antwerp 2022 Legends Sticker Capsule
    7: Austin 2025 Challengers Sticker Capsule
    8: Austin 2025 Contenders Sticker Capsule
    9: Austin 2025 Legends Sticker Capsule

 Total unique items encoded: 50
 Saved item mapping to: item_mapping.pkl


<br>

### Feature Engineering: Add Lag Features

Create `price_lag_1`, `price_lag_2`, and `price_lag_3` features - the prices from 1, 2, and 3 days ago for each item.


In [ ]:
# Create lag features: prices from 1, 2, and 3 days ago PER ITEM
# Sort by item and date to ensure correct order
df_item_daily = df_item_daily.sort_values(['item_name', 'Date'])

# Create lag features for each item
df_item_daily['price_lag_1'] = df_item_daily.groupby('item_name')['price'].shift(1)
df_item_daily['price_lag_2'] = df_item_daily.groupby('item_name')['price'].shift(2)
df_item_daily['price_lag_3'] = df_item_daily.groupby('item_name')['price'].shift(3)

print(f" Created lag features")
print(f"   - price_lag_1 (yesterday's price)")
print(f"   - price_lag_2 (price from 2 days ago)")
print(f"   - price_lag_3 (price from 3 days ago)")
print(f"\n Example for first item (showing lag effect):")
print(df_item_daily[['item_name', 'Date', 'price', 'price_lag_1', 'price_lag_2', 'price_lag_3']].head(10))


✅ Created lag features
   - price_lag_1 (yesterday's price)
   - price_lag_2 (price from 2 days ago)
   - price_lag_3 (price from 3 days ago)

📈 Example for first item (showing lag effect):
              item_name       Date  price  price_lag_1  price_lag_2  \
0  2020 RMR Challengers 2024-10-04  0.207          NaN          NaN   
1  2020 RMR Challengers 2024-10-05  0.219        0.207          NaN   
2  2020 RMR Challengers 2024-10-06  0.224        0.219        0.207   
3  2020 RMR Challengers 2024-10-07  0.209        0.224        0.219   
4  2020 RMR Challengers 2024-10-08  0.203        0.209        0.224   
5  2020 RMR Challengers 2024-10-09  0.194        0.203        0.209   
6  2020 RMR Challengers 2024-10-10  0.204        0.194        0.203   
7  2020 RMR Challengers 2024-10-11  0.212        0.204        0.194   
8  2020 RMR Challengers 2024-10-12  0.223        0.212        0.204   
9  2020 RMR Challengers 2024-10-13  0.234        0.223        0.212   

   price_lag_3  
0          

#### Remove first 3 day 50x3=150 row

In [ ]:
# Check for NaN values from lag feature
print(f" Missing values check:")
print(f" Rows before dropping NaN: {len(df_item_daily):,}")

# Drop rows with NaN (first 3 days per item won't have lag_3)
df_item_daily = df_item_daily.dropna()

print(f" Rows after dropping NaN: {len(df_item_daily):,}")
print(f" Rows removed: {16872 - len(df_item_daily):,} (first 3 days per item)")

 Missing values check:
 Rows before dropping NaN: 16,872
 Rows after dropping NaN: 16,722
 Rows removed: 150 (first 3 days per item)

Clean dataset with lag feature ready!


## Save Final Dataset

Creating CSV with columns:
- `item_encoded` - Numeric ID for item (0-49)
- `price` - Price of the item (target variable)
- `price_lag_1` - Price from 1 day ago (yesterday)
- `price_lag_2` - Price from 2 days ago
- `price_lag_3` - Price from 3 days ago
- `Date` - Date
- `Average_Players` - Player count
- `has_tournament` - Tournament flag (0/1)


In [11]:
# Create final dataset with all lag features
final_df = df_item_daily[['item_encoded', 'price', 'price_lag_1', 'price_lag_2', 'price_lag_3', 
                           'Date', 'Average_Players', 'has_tournament']].copy()

# Save to CSV
final_df.to_csv('csv-files/csgo_item_level_dataset.csv', index=False)

print(" Dataset saved to: csgo_item_level_dataset.csv")
print(f"\n Dataset Info:")
print(f"   Rows: {len(final_df):,}")
print(f"   Columns: {list(final_df.columns)}")
print(f"\n First 10 rows:")
print(final_df.head(10))
print(f"\n Dataset stats:")
print(final_df.describe())


 Dataset saved to: csgo_item_level_dataset.csv

 Dataset Info:
   Rows: 16,722
   Columns: ['item_encoded', 'price', 'price_lag_1', 'price_lag_2', 'price_lag_3', 'Date', 'Average_Players', 'has_tournament']

 First 10 rows:
    item_encoded  price  price_lag_1  price_lag_2  price_lag_3       Date  \
3              0  0.209        0.224        0.219        0.207 2024-10-07   
4              0  0.203        0.209        0.224        0.219 2024-10-08   
5              0  0.194        0.203        0.209        0.224 2024-10-09   
6              0  0.204        0.194        0.203        0.209 2024-10-10   
7              0  0.212        0.204        0.194        0.203 2024-10-11   
8              0  0.223        0.212        0.204        0.194 2024-10-12   
9              0  0.234        0.223        0.212        0.204 2024-10-13   
10             0  0.235        0.234        0.223        0.212 2024-10-14   
11             0  0.228        0.235        0.234        0.223 2024-10-15   
12    